# Gemini API: Function Calling

## Setup

In [1]:
!pip install -U -q google-generativeai

In [2]:
import google.generativeai as genai

In [4]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Single Function


Gemini has the ability to call user-defined functions. Let's take a look at how exactly to do this. Firstly, let us define some functions relating to a hypothetical Italian restaurant located in Berkeley.

In [7]:
def get_full_menu(service: str):
    """List all items on the menu of Gemini's Trattoria for the given service.

    Args:
        name: The type of service, lunch or dinner.
    """
    return ["Chicken Caesar Salad", "Margherita Pizza", "Spaghetti and Meatballs", "Eggplant Parmesan"]


def find_vegetarian_items(items: list[str]):
    """List all dishes in items that are vegetarian.

    Args:
        items: A list of dinner dishes.
    """
    return ["Margherita Pizza", "Eggplant Parmesan"]

def enter_restaurant():
    """You enter Gemini's Trattoria, moving the creaky door."""
    print("The door swings open, making a loud noise.")
    return True

functions = {"get_full_menu": get_full_menu,
             "find_vegetarian_items": find_vegetarian_items,
             "enter_restaurant": enter_restaurant}

After this, we go ahead and define our Gemini model. Notice how we include the argument for tools, which tells the model which functions it has available to use. We create a chat and set automatic function calling to True, which we will touch on later.

In [8]:
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash", tools=functions.values()
)

chat = model.start_chat(enable_automatic_function_calling=True)

Now, we go ahead and send a prompt that requires the model to call our user-defined functions.

In [9]:
response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)
print(response.text)

The dinner menu at Gemini's Trattoria features Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.


As we can see in the chat history, the model initially sends back a function call, to which we automatically respond, which then leads to the final model output.

In [10]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items are on Gemini's Trattoria's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': "The dinner menu at Gemini's Trattoria features Chicken Caesar Salad, Margherita

## Behind The Scenes

In general, the processes goes as follows:
1. The user submits a query to the model.
2. The model responds with a function call.
3. The user runs the function and returns the result of the function.
4. Now, the model will either go back to Step 2 or output a final response, as seen above.

<div>
<img src="https://video.udacity-data.com/topher/2024/June/66749652_function_calling/function_calling.jpeg" width="500"/>
</div>

We can see this step-by-step in action by running the previous example with manual function calling, by setting the automatic function calling argument to False.

In [12]:
chat = model.start_chat(enable_automatic_function_calling=False)

response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)

part = response.candidates[0].content.parts[0]
part

function_call {
  name: "get_full_menu"
  args {
    fields {
      key: "service"
      value {
        string_value: "dinner"
      }
    }
  }
}

After this, we can reply to the model as specified in Step 3, using the functions dictionary we made earlier.

In [13]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
result = functions[part.function_call.name](**part.function_call.args)
s.update({"result": result})

function_response = glm.Part(
    function_response=glm.FunctionResponse(name="get_full_menu", response=s)
)

# Generate the next response
response = chat.send_message(function_response)
print(response.text)

The dinner menu at Gemini's Trattoria includes Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.


## Multiple Function Calls

Our model can also call multiple functions in a row, either at the same time or one after the other, as we see in both cases below.

In [14]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are vegetarian?"
)
print(response.text)

The vegetarian options on Gemini's Trattoria's dinner menu are Margherita Pizza and Eggplant Parmesan.


In [15]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Gemini's Trattoria's dinner menu are vegetarian?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'find_vegetarian_items', 'args': {'items': ['Chicke

In [16]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."
)
print(response.text)

The door swings open, making a loud noise.
I have entered Gemini's Trattoria. The dinner menu features: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan.


In [17]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'enter_restaurant', 'response': {'result': True}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}, 'id': ''}}]
-------------------------------------------------

## Function Calling Config

Using a function calling config lets us control how the Gemini API acts when tools have been specified. Let us use the same previous example and see what happens in each of three cases.

In [18]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

First, we use the NONE mode, which tells the model to not make any function calls. In this example, the model knows about the get_full_menu function but is unable to use it.

In [19]:
tool_config = tool_config_from_mode("none")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are vegetarian?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Gemini's Trattoria's dinner menu are vegetarian?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': "I can't access real-time, live restaurant menus, and menus can change frequently based on season, chef specials, or even daily availability.\n\nTo get the most accurate and up-to-date information, I strongly recommend one of the following:\n\n1.  **Check Gemini's Trattoria's Official Website:** Most restaurants post their current menus online.\n2.  **Call the Restaurant Directly:** This is the best way to confirm vegetarian options and discuss any dietary needs you might have with their staff.\n\n**However, based on typical Italian trattoria offerings, here are common types of items that are usually vegetarian:**\n\n**Antipasti (Appetizers):**\n*   **Bruschetta Pomodoro:** Toasted bread with fresh tomatoe

AUTO mode lets the model decide whether to reply with text or to call specific functions. In this example, we see that the model calls a function to get all menu items but is able to reason on its own on which items are pasta dishes.

In [20]:
tool_config = tool_config_from_mode("auto")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are pasta dishes?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Gemini's Trattoria's dinner menu are pasta dishes?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'I can only list all the items on the dinner menu, not filter for pasta dishes. Would you like to see the full dinner menu?'}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Finally, ANY mode forces the model to make function calls, as seen in the below example. We decide to switch over to the Gemini 1.5 Pro model to achieve the intended behavior.

In [21]:
model = genai.GenerativeModel(
    model_name="gemini-2.5-pro", tools=functions.values()
)

tool_config = tool_config_from_mode("any")
chat = model.start_chat()

response = chat.send_message(
    "Enter Gemini's Trattoria.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Enter Gemini's Trattoria."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


By setting allowed_function_names, the model will only choose from those functions. If it is not set, all of the functions in tools are candidates for function calling. In the below example, since we are in ANY mode, the model is forced to use a function, even though it is not necessarily the best option.

In [22]:
tool_config = tool_config_from_mode("any", ["get_full_menu"])
chat = model.start_chat()

response = chat.send_message(
    "Enter Gemini's Trattoria.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Enter Gemini's Trattoria."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}, 'id': ''}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
